In [50]:
!apt-get update
!apt-get install -y ffmpeg
%pip install pandas datasets[audio] snac transformers torchaudio

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease                 
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 108 not upgraded.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [51]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import torch
from snac import SNAC


model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz")
model.to("cpu")


import torchaudio.transforms as T
from datasets import Audio

def tokenise_audio(waveform, ):
  waveform = torch.from_numpy(waveform).unsqueeze(0)
  waveform = waveform.to(dtype=torch.float32)
  # resample_transform = T.Resample(orig_freq=24000, new_freq=24000)
  # waveform = resample_transform(waveform)
  waveform = waveform.unsqueeze(0).to("cpu")
  #generate the codes from snac
  with torch.inference_mode():
    codes = model.encode(waveform)

  all_codes = []
  for i in range(codes[0].shape[1]):
    all_codes.append(codes[0][0][i].item()+128266)
    all_codes.append(codes[1][0][2*i].item()+128266+4096)
    all_codes.append(codes[2][0][4*i].item()+128266+(2*4096))
    all_codes.append(codes[2][0][(4*i)+1].item()+128266+(3*4096))
    all_codes.append(codes[1][0][(2*i)+1].item()+128266+(4*4096))
    all_codes.append(codes[2][0][(4*i)+2].item()+128266+(5*4096))
    all_codes.append(codes[2][0][(4*i)+3].item()+128266+(6*4096))


  return all_codes

In [52]:
# from transformers import AutoTokenizer
# import os

# tokeniser_length = 128256
# start_of_text = 128000
# end_of_text = 128009

# start_of_speech = tokeniser_length + 1
# end_of_speech = tokeniser_length + 2

# start_of_human = tokeniser_length + 3
# end_of_human = tokeniser_length + 4

# start_of_ai = tokeniser_length + 5
# end_of_ai =  tokeniser_length + 6
# pad_token = tokeniser_length + 7

# audio_tokens_start = tokeniser_length + 10

# tokenizer_name = "canopylabs/orpheus-3b-0.1-pretrained"


# tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
# num_proc = os.cpu_count() - 2

# for example in df:
#     print(example["text"])
#     print(example["codes_list"])

# ds = df.filter(lambda x: x["codes_list"] is not None)
# ds = ds.filter(lambda x: len(x["codes_list"]) > 0)

# #@title Create Input Ids
# def remove_duplicate_frames(example):
#     vals = example["codes_list"]
#     if len(vals) % 7 != 0:
#         raise ValueError("Input list length must be divisible by 7")

#     result = vals[:7]

#     removed_frames = 0

#     for i in range(7, len(vals), 7):
#         current_first = vals[i]
#         previous_first = result[-7]

#         if current_first != previous_first:
#             result.extend(vals[i:i+7])
#         else:
#             removed_frames += 1

#     example["codes_list"] = result

#     return example

# ds = ds.map(remove_duplicate_frames, num_proc=num_proc)

# tok_info = '''*** HERE you can modify the text prompt
# i.e. if you wanted a multispeaker model like canopylabs/orpheus-3b-0.1-ft, you can pass:
# f"{example["source"]}:  {example["text"]}", as is passed.
# '''
# print(tok_info)

# def create_input_ids(example):
#     text_ids = tokenizer.encode(example["text"],  add_special_tokens=True)
#     text_ids.append(end_of_text)
#     example["text_tokens"] = text_ids
#     input_ids = (
#         [start_of_human]
#         + example["text_tokens"]
#         + [end_of_human]
#         + [start_of_ai]
#         + [start_of_speech]
#         + example["codes_list"]
#         + [end_of_speech]
#         + [end_of_ai]
#     )
#     example["input_ids"] = input_ids
#     example["labels"] = input_ids
#     example["attention_mask"] = [1] * len(input_ids)

#     return example

# ds = ds.map(create_input_ids, num_proc=num_proc, remove_columns=["text", "codes_list"])

# ds


In [53]:

from datasets import load_dataset
import pandas as pd
import numpy as np

df = load_dataset("cubbk/audio_swedish_2_dataset_cleaned", split='train', streaming=True, data_dir="8sidor_text_speech_dataset", data_files="jobs_dataset-000011.tar")

tokenised_df_arr = []

for i, dataset_item in enumerate(df):
    if(i > 5):
        break
    
    wf = np.asarray(dataset_item["wav"]["array"], dtype=np.float32)
    codes_list = tokenise_audio(wf)
    tokenised_df_arr.append({
        "codes_list": codes_list,
        "txt": dataset_item["txt"]
    })

tokenized_df = pd.DataFrame(tokenised_df_arr)
print(tokenized_df)

                                          codes_list  \
0  [129980, 135908, 138779, 143131, 147825, 15161...   
1  [131571, 134149, 140434, 142750, 148567, 15025...   
2  [131960, 133191, 138019, 143101, 147991, 14980...   
3  [130233, 133563, 136496, 140878, 146823, 14936...   
4  [130949, 136075, 140077, 143152, 144846, 14997...   
5  [132250, 136030, 140402, 144317, 145842, 14901...   

                                                 txt  
0  Johanna Rytting-Kaneryd spelar i laget Chelsea...  
1  Två kvinnor blev skadade av en explosion på on...  
2  Det är viktigt att de som kommer hem tas om ha...  
3  Det beror på att de köper mer och dyrare mat ä...  
4                         Varning för värme i Europa  
5  Två olika domstolar i FN har sagt att dödandet...  
